# Motif price generator

This code parses TTC item and price tables and computes prices per motif

Copy SLPP.py here for ease of use, source is on github


In [1]:
import re
import sys
from numbers import Number

import six

ERRORS = {
    'unexp_end_string': u'Unexpected end of string while parsing Lua string.',
    'unexp_end_table': u'Unexpected end of table while parsing Lua string.',
    'mfnumber_minus': u'Malformed number (no digits after initial minus).',
    'mfnumber_dec_point': u'Malformed number (no digits after decimal point).',
    'mfnumber_sci': u'Malformed number (bad scientific format).',
}

def sequential(lst):
    length = len(lst)
    if length == 0 or lst[0] != 0:
        return False
    for i in range(length):
        if i + 1 < length:
            if lst[i] + 1 != lst[i+1]:
                return False
    return True


class ParseError(Exception):
    pass


class SLPP(object):

    def __init__(self):
        self.text = ''
        self.ch = ''
        self.at = 0
        self.len = 0
        self.depth = 0
        self.space = re.compile('\s', re.M)
        self.alnum = re.compile('\w', re.M)
        self.newline = '\n'
        self.tab = '\t'

    def decode(self, text):
        if not text or not isinstance(text, six.string_types):
            return
        # FIXME: only short comments removed
        reg = re.compile('--.*$', re.M)
        text = reg.sub('', text, 0)
        self.text = text
        self.at, self.ch, self.depth = 0, '', 0
        self.len = len(text)
        self.next_chr()
        result = self.value()
        return result

    def encode(self, obj):
        self.depth = 0
        return self.__encode(obj)

    def __encode(self, obj):
        s = ''
        tab = self.tab
        newline = self.newline

        if isinstance(obj, str):
            s += '"%s"' % obj.replace(r'"', r'\"')
        elif six.PY2 and isinstance(obj, unicode):
            s += '"%s"' % obj.encode('utf-8').replace(r'"', r'\"')
        elif six.PY3 and isinstance(obj, bytes):
            s += '"{}"'.format(''.join(r'\x{:02x}'.format(c) for c in obj))
        elif isinstance(obj, bool):
            s += str(obj).lower()
        elif obj is None:
            s += 'nil'
        elif isinstance(obj, Number):
            s += str(obj)
        elif isinstance(obj, (list, tuple, dict)):
            self.depth += 1
            if len(obj) == 0 or (not isinstance(obj, dict) and len([
                    x for x in obj
                    if isinstance(x, Number) or (isinstance(x, six.string_types) and len(x) < 10)
               ]) == len(obj)):
                newline = tab = ''
            dp = tab * self.depth
            s += "%s{%s" % (tab * (self.depth - 2), newline)
            if isinstance(obj, dict):
                key = '[%s]' if all(isinstance(k, (int, long)) for k in obj.keys()) else '%s'
                contents = [dp + (key + ' = %s') % (k, self.__encode(v)) for k, v in obj.items()]
                s += (',%s' % newline).join(contents)
            else:
                s += (',%s' % newline).join(
                    [dp + self.__encode(el) for el in obj])
            self.depth -= 1
            s += "%s%s}" % (newline, tab * self.depth)
        return s

    def white(self):
        while self.ch:
            if self.space.match(self.ch):
                self.next_chr()
            else:
                break

    def next_chr(self):
        if self.at >= self.len:
            self.ch = None
            return None
        self.ch = self.text[self.at]
        self.at += 1
        return True

    def value(self):
        self.white()
        if not self.ch:
            return
        if self.ch == '{':
            return self.object()
        if self.ch == "[":
            self.next_chr()
        if self.ch in ['"',  "'",  '[']:
            return self.string(self.ch)
        if self.ch.isdigit() or self.ch == '-':
            return self.number()
        return self.word()

    def string(self, end=None):
        s = ''
        start = self.ch
        if end == '[':
            end = ']'
        if start in ['"',  "'",  '[']:
            while self.next_chr():
                if self.ch == end:
                    self.next_chr()
                    if start != "[" or self.ch == ']':
                        return s
                if self.ch == '\\' and start == end:
                    self.next_chr()
                    if self.ch != end:
                        s += '\\'
                s += self.ch
        raise ParseError(ERRORS['unexp_end_string'])

    def object(self):
        o = {}
        k = None
        idx = 0
        numeric_keys = False
        self.depth += 1
        self.next_chr()
        self.white()
        if self.ch and self.ch == '}':
            self.depth -= 1
            self.next_chr()
            return o  # Exit here
        else:
            while self.ch:
                self.white()
                if self.ch == '{':
                    o[idx] = self.object()
                    idx += 1
                    continue
                elif self.ch == '}':
                    self.depth -= 1
                    self.next_chr()
                    if k is not None:
                        o[idx] = k
                    if len([key for key in o if isinstance(key, six.string_types + (float,  bool, tuple))]) == 0:
                        so = sorted([key for key in o])
                        if sequential(so):
                            ar = []
                            for key in o:
                                ar.insert(key, o[key])
                            o = ar
                    return o  # or here
                else:
                    if self.ch == ',':
                        self.next_chr()
                        continue
                    else:
                        k = self.value()
                        if self.ch == ']':
                            self.next_chr()
                    self.white()
                    ch = self.ch
                    if ch in ('=', ','):
                        self.next_chr()
                        self.white()
                        if ch == '=':
                            o[k] = self.value()
                        else:
                            o[idx] = k
                        idx += 1
                        k = None
        raise ParseError(ERRORS['unexp_end_table'])  # Bad exit here

    words = {'true': True, 'false': False, 'nil': None}
    def word(self):
        s = ''
        if self.ch != '\n':
            s = self.ch
        self.next_chr()
        while self.ch is not None and self.alnum.match(self.ch) and s not in self.words:
            s += self.ch
            self.next_chr()
        return self.words.get(s, s)

    def number(self):
        def next_digit(err):
            n = self.ch
            self.next_chr()
            if not self.ch or not self.ch.isdigit():
                raise ParseError(err)
            return n
        n = ''
        try:
            if self.ch == '-':
                n += next_digit(ERRORS['mfnumber_minus'])
            n += self.digit()
            if n == '0' and self.ch in ['x', 'X']:
                n += self.ch
                self.next_chr()
                n += self.hex()
            else:
                if self.ch and self.ch == '.':
                    n += next_digit(ERRORS['mfnumber_dec_point'])
                    n += self.digit()
                if self.ch and self.ch in ['e', 'E']:
                    n += self.ch
                    self.next_chr()
                    if not self.ch or self.ch not in ('+', '-'):
                        raise ParseError(ERRORS['mfnumber_sci'])
                    n += next_digit(ERRORS['mfnumber_sci'])
                    n += self.digit()
        except ParseError:
            t, e = sys.exc_info()[:2]
            print(e)
            return 0
        try:
            return int(n, 0)
        except:
            pass
        return float(n)

    def digit(self):
        n = ''
        while self.ch and self.ch.isdigit():
            n += self.ch
            self.next_chr()
        return n

    def hex(self):
        n = ''
        while self.ch and (self.ch in 'ABCDEFabcdef' or self.ch.isdigit()):
            n += self.ch
            self.next_chr()
        return n


slpp = SLPP()

__all__ = ['slpp']

Load TTC Files. They contain simple functions and we want just the data structure, so split them on "variable-name=" and keep the stuff on the right, which is the actual Lua table

In [2]:
item_lookup = open(r"C:\Users\jtern\Documents\Elder Scrolls Online\live\AddOns\TamrielTradeCentre\ItemLookUpTable_EN.lua",'r').read()

In [3]:
item_prices = open(r"C:\Users\jtern\Documents\Elder Scrolls Online\live\AddOns\TamrielTradeCentre\PriceTable.lua",'r').read()

In [4]:
prices = slpp.decode(item_prices.split('self.PriceTable=')[-1])

In [5]:
items = slpp.decode(item_lookup.split('self.ItemLookUpTable=')[-1])

In [6]:
# find all items whose name starts with "crafting motif"
motif_keys = [key for key in items.keys() if key.startswith('crafting motif')]

In [7]:
# show an example record. The Lua nested table structure looks like {item_id : {level : {traits : {'Avg' : avg...}}}}
# so we'll need to unnest it
prices['Data'][6444]

{2: {1: {-1: {'Avg': 392.66,
    'Max': 25000,
    'Min': 1,
    'EntryCount': 839,
    'AmountCount': 877,
    'SuggestedPrice': 59.41}}}}

In [8]:
# demonstrate the unnesting process
qual = list(prices['Data'][6444].keys())[0]
level_dict=prices['Data'][6444][qual]
level = list(level_dict.keys())[0]
traits_dict = level_dict[level]
traits = list(traits_dict.keys())[0]
prices_dict = traits_dict[traits]
print(qual,level, traits, prices_dict)

2 1 -1 {'Avg': 392.66, 'Max': 25000, 'Min': 1, 'EntryCount': 839, 'AmountCount': 877, 'SuggestedPrice': 59.41}


In [9]:
#Iterate over all the motifs and build a new list with prices and names. If prices are not found, leave a blank entry
data = []
for motif_key in motif_keys:
    item = items[motif_key]
    entry = {'motif' : motif_key}
    entry['item_id'] = list(item.items())[0][1]
    try:
        price = prices['Data'][entry['item_id']]
    except KeyError:
        data.append(entry)
        continue
    qual = list(price.keys())[0]
    level_dict=price[qual]
    level = list(level_dict.keys())[0]
    traits_dict = level_dict[level]
    traits = list(traits_dict.keys())[0]
    prices_dict = traits_dict[traits]
    entry = {**entry,**prices_dict}
    data.append(entry)

Import Pandas for data manipulation and make a data frame out of our new data

In [10]:
import pandas as pd
#set some pandas display options
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_rows', 1000)

In [11]:
motif_df = pd.DataFrame(data)
motif_df.head()

,motif,item_id,Avg,Max,Min,EntryCount,AmountCount,SuggestedPrice
0,crafting motif 16: glass legs,6328,"19,523.44","28,500.00","5,000.00",14.00,16.00,"17,859.60"
1,crafting motif 17: xivkyn belts,6329,"11,830.23","25,000.00","1,500.00",52.00,52.00,"6,143.88"
2,crafting motif 15: dwemer bows,6340,"3,283.74","20,000.00",500.00,153.00,153.00,"1,184.79"
3,crafting motif 21: ancient orc boots,6349,"36,939.96","50,000.00","5,000.00",25.00,25.00,"28,800.00"
4,crafting motif 19: mercenary axes,6352,"15,536.04","25,000.00","7,619.00",24.00,24.00,"10,261.58"


In [12]:
#extract the motif number from the full name
motif_df['motif_num'] = motif_df['motif'].str.extract("(\d+):",expand=True)

In [13]:
#extract the armor/weapon slot as the last word in the motif name
motif_df['motif_slot'] = motif_df['motif'].str.rsplit(' ',1,expand=True)[1]

In [14]:
# take everything between : and the last word as the motif name
motif_df['motif_name'] = motif_df['motif'].str.rsplit(' ',1,expand=True)[0].str.split(':',expand=True)[1].str.lstrip()

In [15]:
#make a numeric motif number for ordering instead of string, which would give 1,10,11,etc.
#calculate the minimum of Avg and SuggestedPrice as the new calculcated cost
motif_df['motif_sort_num'] = pd.to_numeric(motif_df['motif_num'])
motif_df['calc_cost'] = motif_df[['Avg','SuggestedPrice']].min(axis=1)

In [16]:
# if the motif slot is 'style' then this is the price for the whole book. Fill any missing values with 1m gold
motif_df['Style_Cost']=motif_df.loc[motif_df['motif_slot']=='style']['calc_cost'].fillna(1000000)
# set soul shriven to 0
motif_df.loc[motif_df.motif_name.str.startswith('soul'),'Style_Cost']=0

In [17]:
# rank the page costs by motif in ascending order
motif_df['rank'] = motif_df.sort_values(['motif_sort_num','calc_cost'],ascending=[True,False]).groupby('motif_num')['calc_cost'].rank("dense",ascending=False)

In [18]:
motif_df.sort_values(['motif_sort_num','rank']).head(50)

,motif,item_id,Avg,Max,Min,EntryCount,AmountCount,SuggestedPrice,motif_num,motif_slot,motif_name,motif_sort_num,calc_cost,Style_Cost,rank
60,crafting motif 1: high elf style,6533,576.24,"40,000.00",1.00,922.00,945.00,64.10,1,style,high elf,1,64.10,64.10,1.00
54,crafting motif 2: dark elf style,6511,373.16,"15,000.00",1.00,865.00,886.00,71.44,2,style,dark elf,2,71.44,71.44,1.00
73,crafting motif 3: wood elf style,6580,361.95,"22,000.00",1.00,808.00,819.00,60.68,3,style,wood elf,3,60.68,60.68,1.00
59,crafting motif 4: nord style,6525,678.21,"100,000.00",1.00,761.00,782.00,64.18,4,style,nord,4,64.18,64.18,1.00
61,crafting motif 5: breton style,6534,607.77,"50,000.00",1.00,814.00,849.00,66.12,5,style,breton,5,66.12,66.12,1.00
47,crafting motif 6: redguard style,6500,526.83,"83,000.00",5.00,761.00,773.00,66.75,6,style,redguard,6,66.75,66.75,1.00
72,crafting motif 7: khajiit style,6573,"1,291.24","100,000.00",1.00,780.00,788.00,70.75,7,style,khajiit,7,70.75,70.75,1.00
32,crafting motif 8: orc style,6444,392.66,"25,000.00",1.00,839.00,877.00,59.41,8,style,orc,8,59.41,59.41,1.00
82,crafting motif 9: argonian style,6612,444.95,"30,000.00",1.00,843.00,871.00,66.15,9,style,argonian,9,66.15,66.15,1.00
81,crafting motif 10: imperial style,6610,"133,570.40","1,000,000.00","7,000.00",194.00,194.00,"82,656.32",10,style,imperial,10,"82,656.32","82,656.32",1.00


In [19]:
# we can now do analytics on the costs. show what percentage of slots are the lowest cost, for example.
motif_df.loc[(motif_df['motif_slot']!='style') & (motif_df['rank']==15)]['motif_slot'].value_counts(normalize=True)

maces       0.39
belts       0.26
daggers     0.17
axes        0.04
gloves      0.04
shoulders   0.04
boots       0.04
Name: motif_slot, dtype: float64

In [20]:
# sum up the total motif cost (calc_cost)
by_page_df = motif_df.loc[motif_df['motif_slot'] != 'style'].groupby(['motif_sort_num','motif_name'])['calc_cost'].sum()

In [21]:
# create a new data frame with just prices for full styles
by_style_df = motif_df.loc[motif_df['motif_slot']=='style']

In [22]:
by_style_df.head()

,motif,item_id,Avg,Max,Min,EntryCount,AmountCount,SuggestedPrice,motif_num,motif_slot,motif_name,motif_sort_num,calc_cost,Style_Cost,rank
6,crafting motif 12: barbaric style,6359,"2,281.21","200,000.00",1.00,699.00,729.00,268.74,12,style,barbaric,12,268.74,268.74,1.00
7,crafting motif 14: daedric style,6362,"4,153.27","98,888.00",3.00,477.00,482.00,757.92,14,style,daedric,14,757.92,757.92,1.00
26,crafting motif 13: primal style,6431,"1,709.13","150,000.00",1.00,884.00,917.00,207.90,13,style,primal,13,207.90,207.90,1.00
32,crafting motif 8: orc style,6444,392.66,"25,000.00",1.00,839.00,877.00,59.41,8,style,orc,8,59.41,59.41,1.00
47,crafting motif 6: redguard style,6500,526.83,"83,000.00",5.00,761.00,773.00,66.75,6,style,redguard,6,66.75,66.75,1.00


In [23]:
# create a new cost df by joining by_page and by_style on motif_sort_num. Use outer join because some styles don't have pages and vice-versa
cost_df = by_page_df.reset_index().merge(by_style_df[['motif_sort_num','Style_Cost','motif_name']],on='motif_sort_num',how='outer')

In [24]:
cost_df.head()

,motif_sort_num,motif_name_x,calc_cost,Style_Cost,motif_name_y
0,15,dwemer,"15,751.08","1,000,000.00",dwemer
1,16,glass,"305,943.03","175,664.40",glass
2,17,xivkyn,"132,927.32","1,245,000.00",xivkyn
3,18,akaviri,"491,858.95","1,000,000.00",akaviri
4,19,mercenary,"407,891.40","1,500,000.00",mercenary


In [25]:
# compute the lesser of the summed page cost and style cost
cost_df['min_cost'] = cost_df[['calc_cost','Style_Cost']].min(axis=1)

In [26]:
# add a cumulative cost of buying the cheapest motifs in order
cost_df['cumulative_cost'] = cost_df.sort_values('min_cost')['min_cost'].cumsum()

In [27]:
# take the two motif names from the outer join and make a single non-null motif name
cost_df['motif_name'] = cost_df.apply(lambda r : r['motif_name_x'] if pd.notnull(r['motif_name_x']) else r['motif_name_y'],axis=1)

In [28]:
#drop the extra name columns and make names friendlier
new_cost_df = cost_df.drop(columns=['motif_name_x','motif_name_y']).rename(columns={'calc_cost':'ByPage_Cost', 'motif_sort_num':'motif_number'}).sort_values('min_cost').reset_index(drop=True)

In [29]:
# hide prices where the book is more expensive than the by_page cost
import numpy as np
new_cost_df['Style_Cost'] = new_cost_df.apply(lambda r : np.nan if (pd.notna(r['ByPage_Cost']) and (r['ByPage_Cost']<r['Style_Cost'])) else r['Style_Cost'],axis=1)

In [30]:
new_cost_df.fillna('')

,motif_number,ByPage_Cost,Style_Cost,min_cost,cumulative_cost,motif_name
0,29,,0.00,0.00,0.00,soul shriven
1,8,,59.41,59.41,59.41,orc
2,3,,60.68,60.68,120.09,wood elf
3,1,,64.10,64.10,184.19,high elf
4,4,,64.18,64.18,248.37,nord
5,5,,66.12,66.12,314.49,breton
6,9,,66.15,66.15,380.64,argonian
7,6,,66.75,66.75,447.39,redguard
8,7,,70.75,70.75,518.14,khajiit
9,2,,71.44,71.44,589.58,dark elf


In [31]:
# Format the numeric columns nicely
for col in ['cumulative_cost','min_cost','ByPage_Cost','Style_Cost']:
    new_cost_df[col] = new_cost_df[col].apply(lambda x : "{:,.2f}".format(x) if pd.notna(x) else '')
new_cost_df.reset_index(inplace=True)
new_cost_df.head()


,index,motif_number,ByPage_Cost,Style_Cost,min_cost,cumulative_cost,motif_name
0,0,29,,0.00,0.00,0.00,soul shriven
1,1,8,,59.41,59.41,59.41,orc
2,2,3,,60.68,60.68,120.09,wood elf
3,3,1,,64.10,64.10,184.19,high elf
4,4,4,,64.18,64.18,248.37,nord


In [32]:
new_cost_df.fillna('').head(50)

,index,motif_number,ByPage_Cost,Style_Cost,min_cost,cumulative_cost,motif_name
0,0,29,,0.00,0.00,0.00,soul shriven
1,1,8,,59.41,59.41,59.41,orc
2,2,3,,60.68,60.68,120.09,wood elf
3,3,1,,64.10,64.10,184.19,high elf
4,4,4,,64.18,64.18,248.37,nord
5,5,5,,66.12,66.12,314.49,breton
6,6,9,,66.15,66.15,380.64,argonian
7,7,6,,66.75,66.75,447.39,redguard
8,8,7,,70.75,70.75,518.14,khajiit
9,9,2,,71.44,71.44,589.58,dark elf


In [33]:
# Use StringIO to write the data frame in a reddit-friendly manner (pipe delimited)
from io import StringIO

In [34]:
buf = StringIO()
new_cost_df.to_csv(buf,index=False,sep='|')

In [35]:
# reset the buffer to start, and read the buffer into a string
buf.seek(0)

0

In [36]:
print(buf.readline().strip())
print(':--|--:|--:|--:|--:|--:|:--')
for line in buf.readlines():
    print(line.strip())

index|motif_number|ByPage_Cost|Style_Cost|min_cost|cumulative_cost|motif_name
:--|--:|--:|--:|--:|--:|:--
0|29||0.00|0.00|0.00|soul shriven
1|8||59.41|59.41|59.41|orc
2|3||60.68|60.68|120.09|wood elf
3|1||64.10|64.10|184.19|high elf
4|4||64.18|64.18|248.37|nord
5|5||66.12|66.12|314.49|breton
6|9||66.15|66.15|380.64|argonian
7|6||66.75|66.75|447.39|redguard
8|7||70.75|70.75|518.14|khajiit
9|2||71.44|71.44|589.58|dark elf
10|13||207.90|207.90|797.48|primal
11|12||268.74|268.74|1,066.22|barbaric
12|11||278.51|278.51|1,344.73|ancient elf
13|14||757.92|757.92|2,102.65|daedric
14|31|12,254.49||12,254.49|14,357.14|skinchanger
15|15|15,751.08||15,751.08|30,108.22|dwemer
16|81|16,359.24||16,359.24|46,467.46|new moon priest
17|28|24,558.32||24,558.32|71,025.78|ra gada
18|84|28,070.95||28,070.95|99,096.73|blackreach vanguard
19|57|44,454.44||44,454.44|143,551.17|ebonshadow
20|33|47,440.46||47,440.46|190,991.63|thieves guild
21|48|47,914.28||47,914.28|238,905.91|ashlander
22|63|49,712.14||49,712.1

In [77]:
print(s)

NameError: name 's' is not defined

In [503]:
new_items = []
for _item,_item_dict in items.items():
    _item_entry = {'name' : _item}
    if isinstance(_item_dict, dict) :
        _item_entry['type'] = list(_item_dict.keys())[0]
        _item_entry['ttc_id'] = _item_dict[list(_item_dict.keys())[0]]
    else:
        _item_entry['type'] = None
        _item_entry['ttc_id'] = _item_dict[0]
    #_prices_sub = prices['Data'][item_entry['ttc_id']]
    new_items.append(_item_entry)
        
    

In [501]:
prices['Data'][183]

{2: {1: {-1: {'Avg': 1034.25,
    'Max': 2500,
    'Min': 180,
    'EntryCount': 8,
    'AmountCount': 8}}}}

In [506]:
items_new_df = pd.DataFrame(new_items)

In [525]:
items_new_df.sort_values('type').groupby('type').first()

,name,ttc_id
type,,
1.00,roast pig,1980
2.00,congealed melon jelly,7001
3.00,cold jerall view inn carrot cake,6822
4.00,melon-baked parmesan pork,5187
5.00,garlic cod with potato crust,6437
6.00,late-summer corn slaw,6236
7.00,dawnstar sun's dusk chowder,5593
8.00,stale bread,14740
20.00,flat nut brown ale,11778


In [508]:
items_new_df['type'].unique()

array([4.500e+02, 3.000e+02, 2.100e+01, 1.250e+03, 9.500e+02, 2.500e+02,
             nan, 2.400e+01, 2.300e+01, 1.710e+02, 2.000e+00, 1.650e+03,
       1.020e+02, 1.700e+02, 1.000e+03, 2.600e+01, 6.000e+00, 1.000e+02,
       1.700e+03, 7.000e+00, 5.000e+00, 2.200e+01, 1.500e+02, 7.500e+02,
       4.000e+00, 2.300e+03, 1.750e+03, 2.000e+01, 1.040e+02, 1.900e+03,
       4.300e+01, 1.030e+02, 1.450e+03, 2.050e+03, 2.400e+03, 4.040e+02,
       1.500e+03, 9.000e+02, 1.000e+00, 2.500e+01, 4.000e+02, 1.510e+02,
       3.000e+00, 1.600e+03, 4.010e+02, 2.350e+03, 1.550e+03, 2.000e+03,
       4.070e+02, 2.450e+03, 1.950e+03, 4.200e+01, 1.800e+03, 4.700e+01,
       5.000e+02, 4.000e+01, 8.500e+02, 4.100e+01, 4.500e+01, 2.100e+03,
       1.850e+03, 8.000e+02, 4.020e+02, 4.400e+01, 4.060e+02, 4.600e+01,
       2.150e+03, 4.800e+01, 2.700e+01, 6.100e+01, 8.000e+00, 6.000e+01,
       2.550e+03, 2.600e+03, 1.520e+02, 2.650e+03, 1.400e+03, 8.000e+01,
       1.460e+03, 1.660e+03, 1.760e+03, 1.465e+03, 

In [36]:
new_cost_df

,index,motif_number,ByPage_Cost,Style_Cost,min_cost,cumulative_cost,motif_name
0,0,29,,0.00,0.00,0.00,soul shriven
1,1,7,,42.23,42.23,42.23,khajiit
2,2,8,,45.83,45.83,88.06,orc
3,3,2,,46.27,46.27,134.33,dark elf
4,4,9,,47.66,47.66,181.99,argonian
5,5,4,,48.13,48.13,230.12,nord
6,6,5,,52.14,52.14,282.26,breton
7,7,1,,52.17,52.17,334.43,high elf
8,8,3,,52.19,52.19,386.62,wood elf
9,9,6,,54.23,54.23,440.85,redguard


In [32]:
by_style_df

,motif,item_id,Avg,Max,Min,EntryCount,AmountCount,SuggestedPrice,motif_num,motif_slot,motif_name,motif_sort_num,calc_cost,Style_Cost,rank
6,crafting motif 12: barbaric style,6359,367.33,"100,000.00",1.00,"1,351.00","1,377.00",330.60,12,style,barbaric,12,330.60,330.60,1.00
7,crafting motif 14: daedric style,6362,"1,164.49","35,000.00",99.00,752.00,758.00,"1,048.04",14,style,daedric,14,"1,048.04","1,048.04",1.00
26,crafting motif 13: primal style,6431,384.03,"20,000.00",50.00,"1,713.00","1,732.00",345.62,13,style,primal,13,345.62,345.62,1.00
32,crafting motif 8: orc style,6444,103.74,"30,000.00",1.00,"1,405.00","1,444.00",93.36,8,style,orc,8,93.36,93.36,1.00
47,crafting motif 6: redguard style,6500,102.95,"50,000.00",5.00,"1,355.00","1,387.00",92.65,6,style,redguard,6,92.65,92.65,1.00
49,crafting motif 11: ancient elf style,6503,419.71,"90,000.00",1.00,"1,235.00","1,245.00",377.74,11,style,ancient elf,11,377.74,377.74,1.00
54,crafting motif 2: dark elf style,6511,98.89,"15,000.00",2.13,"1,392.00","1,473.00",89.00,2,style,dark elf,2,89.00,89.00,1.00
59,crafting motif 4: nord style,6525,101.31,"800,000.00",1.00,"1,498.00","1,564.00",91.18,4,style,nord,4,91.18,91.18,1.00
60,crafting motif 1: high elf style,6533,94.58,"13,000.00",1.00,"1,533.00","1,578.00",85.12,1,style,high elf,1,85.12,85.12,1.00
61,crafting motif 5: breton style,6534,93.95,"20,000.00",3.33,"1,502.00","1,568.00",84.56,5,style,breton,5,84.56,84.56,1.00


In [33]:
by_page_df

motif_sort_num  motif_name          
15               dwemer                 22,371.85
16               glass                  59,407.39
17               xivkyn                 74,860.60
18               akaviri               196,645.56
19               mercenary              70,546.80
20               yokudan               113,625.88
21               ancient orc           163,903.78
22               trinimac              126,611.02
23               malacath               85,298.66
24               outlaw                 95,180.07
25               aldmeri dominion       53,152.36
26               daggerfall covenant    61,110.74
27               ebonheart pact         71,938.35
28               ra gada                18,010.96
30               morag tong             91,644.96
31               skinchanger            25,098.53
32               abah's watch           81,424.26
33               thieves guild          17,275.89
34               assassins league       98,018.47
35           

In [35]:
by_page_df

motif_sort_num  motif_name          
15               dwemer                 24,857.62
16               glass                  66,008.20
17               xivkyn                 83,178.44
18               akaviri               218,495.07
19               mercenary              78,385.33
20               yokudan               126,250.97
21               ancient orc           182,115.31
22               trinimac              140,678.92
23               malacath               94,776.30
24               outlaw                105,755.64
25               aldmeri dominion       59,058.15
26               daggerfall covenant    67,900.82
27               ebonheart pact         79,931.51
28               ra gada                20,012.18
30               morag tong            101,827.75
31               skinchanger            27,887.24
32               abah's watch           90,471.41
33               thieves guild          19,195.42
34               assassins league      108,909.39
35           

In [32]:
from io import StringIO
buf = StringIO()

In [33]:
motif_df.loc[motif_df.motif_name==' moongrave fane'][['motif','Min','Avg','Max','SuggestedPrice','AmountCount']].to_csv(buf,sep='|',index=False)

In [38]:
motif_df.motif_name.unique()

array([' glass', ' xivkyn', ' dwemer', ' ancient orc', ' mercenary',
       ' akaviri', ' barbaric', ' daedric', ' primal', ' orc',
       ' redguard', ' ancient elf', ' dark elf', ' nord', ' high elf',
       ' breton', ' khajiit', ' wood elf', ' imperial', ' argonian',
       ' ebonheart pact', ' trinimac', ' daggerfall covenant', ' outlaw',
       ' aldmeri dominion', ' malacath', ' thieves guild',
       ' soul shriven', ' assassins league', " abah's watch", ' minotaur',
       " dro-m'athra", ' order hour', ' dark brotherhood', ' celestial',
       ' yokudan', ' draugr', ' skinchanger', ' hollowjack',
       ' silken ring', ' mazzatun', ' ra gada', ' ebony', ' morag tong',
       ' militant ordinator', ' buoyant armiger', ' ashlander',
       ' telvanni', ' hlaalu', ' redoran', ' ebonshadow', ' dreadhorn',
       ' apostle', ' bloodforge', ' worm cult', ' psijic', ' sapiarch',
       ' pyandonean', ' fang lair', ' scalecaller', ' welkynar',
       ' dremora', ' dead-water', ' silv

In [37]:
buf.seek(0)

0

In [38]:
print(buf.read())

index|motif_number|ByPage_Cost|Style_Cost|min_cost|cumulative_cost|motif_name
0|29||0.00|0.00|0.00|soul shriven
1|7||42.23|42.23|42.23|khajiit
2|8||45.83|45.83|88.06|orc
3|2||46.27|46.27|134.33|dark elf
4|9||47.66|47.66|181.99|argonian
5|4||48.13|48.13|230.12|nord
6|5||52.14|52.14|282.26|breton
7|1||52.17|52.17|334.43|high elf
8|3||52.19|52.19|386.62|wood elf
9|6||54.23|54.23|440.85|redguard
10|13||174.62|174.62|615.47|primal
11|12||194.19|194.19|809.66|barbaric
12|11||203.34|203.34|1,013.00|ancient elf
13|14||423.44|423.44|1,436.44|daedric
14|15|11,879.58||11,879.58|13,316.02|dwemer
15|28|13,995.88||13,995.88|27,311.90|ra gada
16|81|22,439.10||22,439.10|49,751.00|new moon priest
17|84|24,681.21||24,681.21|74,432.21|blackreach vanguard
18|62|25,804.99||25,804.99|100,237.20|sapiarch
19|33|34,781.04||34,781.04|135,018.24|thieves guild
20|63|51,514.26||51,514.26|186,532.50|dremora
21|25|55,006.50||55,006.50|241,539.00|aldmeri dominion
22|88|58,279.55||58,279.55|299,818.55|ancestral orc
23

In [ ]:
for col in ['Min','Avg','Max','SuggestedPrice']:
    new_cost_df[col] = new_cost_df[col].apply(lambda x : "{:,.2f}".format(x))


In [39]:
buf.seek(0)
print(buf.readline().strip())
print(':--|:--|--:|--:|--:|:--')
for line in buf.readlines():
    print(line.strip())

index|motif_number|ByPage_Cost|Style_Cost|min_cost|cumulative_cost|motif_name
:--|:--|--:|--:|--:|:--
0|29||0.00|0.00|0.00|soul shriven
1|7||42.23|42.23|42.23|khajiit
2|8||45.83|45.83|88.06|orc
3|2||46.27|46.27|134.33|dark elf
4|9||47.66|47.66|181.99|argonian
5|4||48.13|48.13|230.12|nord
6|5||52.14|52.14|282.26|breton
7|1||52.17|52.17|334.43|high elf
8|3||52.19|52.19|386.62|wood elf
9|6||54.23|54.23|440.85|redguard
10|13||174.62|174.62|615.47|primal
11|12||194.19|194.19|809.66|barbaric
12|11||203.34|203.34|1,013.00|ancient elf
13|14||423.44|423.44|1,436.44|daedric
14|15|11,879.58||11,879.58|13,316.02|dwemer
15|28|13,995.88||13,995.88|27,311.90|ra gada
16|81|22,439.10||22,439.10|49,751.00|new moon priest
17|84|24,681.21||24,681.21|74,432.21|blackreach vanguard
18|62|25,804.99||25,804.99|100,237.20|sapiarch
19|33|34,781.04||34,781.04|135,018.24|thieves guild
20|63|51,514.26||51,514.26|186,532.50|dremora
21|25|55,006.50||55,006.50|241,539.00|aldmeri dominion
22|88|58,279.55||58,279.55|299

In [36]:
buf.readline()

'|motif_number|ByPage_Cost|Style_Cost|min_cost|cumulative_cost|motif_name\r\n'

In [61]:
motif_df.loc[motif_df.motif_name.str.startswith(' soul')]

,motif,item_id,Avg,Max,Min,EntryCount,AmountCount,SuggestedPrice,motif_num,motif_slot,motif_name,motif_sort_num,calc_cost,Style_Cost,rank
186,crafting motif 29: soul shriven style,7949,nan,nan,nan,nan,nan,nan,29,style,soul shriven,29,nan,"1,000,000.00",nan


In [84]:
[key for key in items.keys() if 'legion' in key]

['mark of the legion',
 'boots of the seventh legion',
 'fifth legion porter',
 'recipe: fifth legion porter',
 "seventh legion's necklace",
 "seventh legion's axe",
 "seventh legion's greaves",
 "seventh legion's conjured oath-band",
 "seventh legion's girdle",
 "seventh legion's shield",
 'seventh legion gauntlets',
 "seventh legion's maul",
 "seventh legion's helm",
 "seventh legion's sabatons",
 "seventh legion's cuirass",
 "seventh legion's restoration staff",
 "seventh legion's ring",
 "seventh legion's ice staff",
 "seventh legion's the wolfguard",
 "seventh legion's lightning staff",
 "seventh legion's dagger",
 "seventh legion's battle axe",
 "seventh legion's sword",
 "seventh legion's bow",
 "seventh legion's pauldron",
 "seventh legion's mace",
 "seventh legion's lava-charred greaves",
 "seventh legion's gauntlets",
 "seventh legion's ayleid breastplate",
 "seventh legion's greatsword",
 "seventh legion's inferno staff",
 "ex-legionairy's breastplate of soulshine",
 'might 

In [45]:
last_prices = """
rank|motif_number|By Page|By Book|Best|Cumulative|motif_name|Prior|Change
0|29||0.00|0.00|0.00|soul shriven|0|+0
1|7||42.23|42.23|42.23|khajiit|2|-1
2|8||45.83|45.83|88.06|orc|1|+1
3|2||46.27|46.27|134.33|dark elf|8|-5
4|9||47.66|47.66|181.99|argonian|3|+1
5|4||48.13|48.13|230.12|nord|9|-4
6|5||52.14|52.14|282.26|breton|4|+2
7|1||52.17|52.17|334.43|high elf|7|+0
8|3||52.19|52.19|386.62|wood elf|5|+3
9|6||54.23|54.23|440.85|redguard|6|+3
10|13||174.62|174.62|615.47|primal|10|+0
11|12||194.19|194.19|809.66|barbaric|11|+0
12|11||203.34|203.34|1,013.00|ancient elf|12|+0
13|14||423.44|423.44|1,436.44|daedric|13|+0
14|15|11,879.58||11,879.58|13,316.02|dwemer|14|+0
15|28|13,995.88||13,995.88|27,311.90|ra gada|15|+0
16|81|22,439.10||22,439.10|49,751.00|new moon priest|18|-2
17|84|24,681.21||24,681.21|74,432.21|blackreach vanguard|19|-2
18|62|25,804.99||25,804.99|100,237.20|sapiarch|17|+1
19|33|34,781.04||34,781.04|135,018.24|thieves guild|20|-1
20|63|51,514.26||51,514.26|186,532.50|dremora|46|-26
21|25|55,006.50||55,006.50|241,539.00|aldmeri dominion|22|-1
22|88|58,279.55||58,279.55|299,818.55|ancestral orc|24|-2
23|87|60,574.14||60,574.14|360,392.69|ancestral nord|28|-5
24|10||61,244.52|61,244.52|421,637.21|imperial|26|-2
25|31|62,741.42||62,741.42|484,378.63|skinchanger|21|+4
26|73|68,249.41||68,249.41|552,628.04|anequina|31|-5
27|17|68,745.45||68,745.45|621,373.49|xivkyn|30|-3
28|57|69,215.81||69,215.81|690,589.30|ebonshadow|29|-1
29|27|70,395.31||70,395.31|760,984.61|ebonheart pact|23|+6
30|89|70,993.93||70,993.93|831,978.54|ancestral high elf|32|-2
31|26|76,271.50||76,271.50|908,250.04|daggerfall covenant|25|+6
32|61|84,099.23||84,099.23|992,349.27|psijic|33|-1
33|75|84,969.54||84,969.54|1,077,318.81|sunspire|34|-1
34|38|86,473.69||86,473.69|1,163,792.50|draugr|35|-1
35|48|86,630.91||86,630.91|1,250,423.41|ashlander|27|+8
36|16|138,567.21|97,838.49|97,838.49|1,348,261.90|glass|36|+0
37|41|98,503.15||98,503.15|1,446,765.05|celestial|37|+0
38|36|104,394.45||104,394.45|1,551,159.50|dark brotherhood|44|-6
39|85|107,518.29||107,518.29|1,658,677.79|greymoor|53|-14
40|54|107,869.06||107,869.06|1,766,546.85|bloodforge|38|+2
41|55|108,568.43||108,568.43|1,875,115.28|dreadhorn|39|+2
42|76|129,274.36||129,274.36|2,004,389.64|dragonguard|50|-8
43|51|132,068.15||132,068.15|2,136,457.79|hlaalu|45|-2
44|52|132,712.77||132,712.77|2,269,170.56|redoran|43|+1
45|50|138,900.33||138,900.33|2,408,070.89|telvanni|41|+4
46|69|139,043.48||139,043.48|2,547,114.37|dead-water|49|-3
47|44|141,991.11||141,991.11|2,689,105.48|silken ring|47|+0
48|64|143,414.15||143,414.15|2,832,519.63|pyandonean|40|+8
49|67|144,111.34||144,111.34|2,976,630.97|welkynar|51|-2
50|37|232,342.00|145,730.70|145,730.70|3,122,361.67|ebony|48|+2
51|70|146,511.44||146,511.44|3,268,873.11|elder argonian|42|+9
52|59|148,055.70||148,055.70|3,416,928.81|scalecaller|52|+0
53|20|169,654.78||169,654.78|3,586,583.59|yokudan|57|-4
54|23|173,677.77||173,677.77|3,760,261.36|malacath|55|-1
55|24|178,110.02||178,110.02|3,938,371.38|outlaw|56|-1
56|32|179,117.82||179,117.82|4,117,489.20|abah's watch|54|+2
57|42|187,396.76||187,396.76|4,304,885.96|hollowjack|16|+41
58|19|192,682.24||192,682.24|4,497,568.20|mercenary|58|+0
59|34|209,781.05||209,781.05|4,707,349.25|assassins league|59|+0
60|74|239,378.90||239,378.90|4,946,728.15|pellitine|62|-2
61|30|269,118.19||269,118.19|5,215,846.34|morag tong|60|+1
62|56|276,857.75||276,857.75|5,492,704.09|apostle|65|-3
63|65|283,436.91||283,436.91|5,776,141.00|huntsman|61|+2
64|21|283,886.55||283,886.55|6,060,027.55|ancient orc|63|+1
65|40|330,000.42||330,000.42|6,390,027.97|order hour|64|+1
66|22|332,270.34||332,270.34|6,722,298.31|trinimac|69|-3
67|45|353,795.19||353,795.19|7,076,093.50|mazzatun|68|-1
68|79|357,895.67||357,895.67|7,433,989.17|refabricated|66|+2
69|58|375,647.42||375,647.42|7,809,636.59|fang lair|70|-1
70|60|450,989.59||450,989.59|8,260,626.18|worm cult|67|+3
71|72|463,215.66||463,215.66|8,723,841.84|meridian|74|-3
72|80|474,047.50||474,047.50|9,197,889.34|shield of senchal|71|+1
73|96|505,011.33||505,011.33|9,702,900.67|arkthzand armory|NEW|NEW
74|78|524,360.09||524,360.09|10,227,260.76|moongrave fane|76|-2
75|66|553,093.19||553,093.19|10,780,353.95|silver dawn|73|+2
76|47|553,656.17||553,656.17|11,334,010.12|buoyant armiger|75|+1
77|71|589,298.53||589,298.53|11,923,308.65|coldsnap|77|+0
78|18|632,303.27||632,303.27|12,555,611.92|akaviri|72|+6
79|68|637,689.44||637,689.44|13,193,301.36|honor guard|80|-1
80|86|665,171.41||665,171.41|13,858,472.77|sea giant|83|-3
81|82|731,143.28||731,143.28|14,589,616.05|icereach coven|81|+0
82|35|732,159.75||732,159.75|15,321,775.80|dro-m'athra|82|+0
83|83|750,990.84||750,990.84|16,072,766.64|pyre watch|78|+5
84|49|835,804.64||835,804.64|16,908,571.28|militant ordinator|84|+0
85|77|934,583.94||934,583.94|17,843,155.22|stags of z'en|79|+6
86|39|1,136,787.21|1,000,000.00|1,000,000.00|18,843,155.22|minotaur|85|+1
87|94|2,673,204.60||2,673,204.60|21,516,359.82|ancestral reach|NEW|NEW"""

In [46]:
last_price_df = pd.read_csv(StringIO(last_prices),delimiter='|')

In [47]:
last_price_df

,rank,motif_number,By Page,By Book,Best,Cumulative,motif_name,Prior,Change
0,0,29,NaN,0.00,0.00,0.00,soul shriven,0,+0
1,1,7,NaN,42.23,42.23,42.23,khajiit,2,-1
2,2,8,NaN,45.83,45.83,88.06,orc,1,+1
3,3,2,NaN,46.27,46.27,134.33,dark elf,8,-5
4,4,9,NaN,47.66,47.66,181.99,argonian,3,+1
5,5,4,NaN,48.13,48.13,230.12,nord,9,-4
6,6,5,NaN,52.14,52.14,282.26,breton,4,+2
7,7,1,NaN,52.17,52.17,334.43,high elf,7,+0
8,8,3,NaN,52.19,52.19,386.62,wood elf,5,+3
9,9,6,NaN,54.23,54.23,440.85,redguard,6,+3


In [40]:
new_cost_df

,index,motif_number,ByPage_Cost,Style_Cost,min_cost,cumulative_cost,motif_name
0,0,29,,0.00,0.00,0.00,soul shriven
1,1,8,,59.41,59.41,59.41,orc
2,2,3,,60.68,60.68,120.09,wood elf
3,3,1,,64.10,64.10,184.19,high elf
4,4,4,,64.18,64.18,248.37,nord
5,5,5,,66.12,66.12,314.49,breton
6,6,9,,66.15,66.15,380.64,argonian
7,7,6,,66.75,66.75,447.39,redguard
8,8,7,,70.75,70.75,518.14,khajiit
9,9,2,,71.44,71.44,589.58,dark elf


In [48]:
diff_df = new_cost_df.merge(last_price_df[['rank','motif_name']],how='outer',on='motif_name')

In [49]:
diff_df

,index,motif_number,ByPage_Cost,Style_Cost,min_cost,cumulative_cost,motif_name,rank
0,0,29,,0.00,0.00,0.00,soul shriven,0.00
1,1,8,,59.41,59.41,59.41,orc,2.00
2,2,3,,60.68,60.68,120.09,wood elf,8.00
3,3,1,,64.10,64.10,184.19,high elf,7.00
4,4,4,,64.18,64.18,248.37,nord,5.00
5,5,5,,66.12,66.12,314.49,breton,6.00
6,6,9,,66.15,66.15,380.64,argonian,4.00
7,7,6,,66.75,66.75,447.39,redguard,9.00
8,8,7,,70.75,70.75,518.14,khajiit,1.00
9,9,2,,71.44,71.44,589.58,dark elf,3.00


In [50]:
diff_df['change']=diff_df.apply(lambda r : '{0:+d}'.format(r['index']-int(r['rank'])) if pd.notna(r['rank']) else np.NaN,axis=1)

In [51]:
diff_df.rename(columns={'rank':'Prior','index':'Rank'},inplace=True)

In [52]:
diff_df['Prior'] = diff_df['Prior'].apply(lambda v : str(int(v)) if pd.notna(v) else 'NEW')
diff_df['change'] = diff_df['change'].apply(lambda v : str(v) if pd.notna(v) else 'NEW')

In [53]:
diff_df

,Rank,motif_number,ByPage_Cost,Style_Cost,min_cost,cumulative_cost,motif_name,Prior,change
0,0,29,,0.00,0.00,0.00,soul shriven,0,+0
1,1,8,,59.41,59.41,59.41,orc,2,-1
2,2,3,,60.68,60.68,120.09,wood elf,8,-6
3,3,1,,64.10,64.10,184.19,high elf,7,-4
4,4,4,,64.18,64.18,248.37,nord,5,-1
5,5,5,,66.12,66.12,314.49,breton,6,-1
6,6,9,,66.15,66.15,380.64,argonian,4,+2
7,7,6,,66.75,66.75,447.39,redguard,9,-2
8,8,7,,70.75,70.75,518.14,khajiit,1,+7
9,9,2,,71.44,71.44,589.58,dark elf,3,+6


In [54]:
buf = StringIO()
colnames = ['Rank','Motif Num','By Page', 'By Book', 'Best', 'Cumulative','Motif','Prior','Change']
diff_df.to_csv(buf,index=False,sep='|',header=colnames)
buf.seek(0)
print(buf.readline().strip())
print(':--|:--|--:|--:|--:|:--|:--|:--|:--')
for line in buf.readlines():
    print(line.strip())

Rank|Motif Num|By Page|By Book|Best|Cumulative|Motif|Prior|Change
:--|:--|--:|--:|--:|:--|:--|:--|:--
0|29||0.00|0.00|0.00|soul shriven|0|+0
1|8||59.41|59.41|59.41|orc|2|-1
2|3||60.68|60.68|120.09|wood elf|8|-6
3|1||64.10|64.10|184.19|high elf|7|-4
4|4||64.18|64.18|248.37|nord|5|-1
5|5||66.12|66.12|314.49|breton|6|-1
6|9||66.15|66.15|380.64|argonian|4|+2
7|6||66.75|66.75|447.39|redguard|9|-2
8|7||70.75|70.75|518.14|khajiit|1|+7
9|2||71.44|71.44|589.58|dark elf|3|+6
10|13||207.90|207.90|797.48|primal|10|+0
11|12||268.74|268.74|1,066.22|barbaric|11|+0
12|11||278.51|278.51|1,344.73|ancient elf|12|+0
13|14||757.92|757.92|2,102.65|daedric|13|+0
14|31|12,254.49||12,254.49|14,357.14|skinchanger|25|-11
15|15|15,751.08||15,751.08|30,108.22|dwemer|14|+1
16|81|16,359.24||16,359.24|46,467.46|new moon priest|16|+0
17|28|24,558.32||24,558.32|71,025.78|ra gada|15|+2
18|84|28,070.95||28,070.95|99,096.73|blackreach vanguard|17|+1
19|57|44,454.44||44,454.44|143,551.17|ebonshadow|28|-9
20|33|47,440.46||4